In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

In [2]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input
from keras.models import Model
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


In [71]:

imgs = np.load('75/images.npy')
imgs = imgs.astype('float32')
indices = np.random.permutation(imgs.shape[0])
imgs = imgs[indices]

split_1 = int(18000*0.8)
split_2 = int(18000*0.9)

train_imgs = imgs[:split_1]
val_imgs = imgs[split_1:split_2]
test_imgs = imgs[split_2:]

train_imgs = train_imgs / 255.0
test_imgs = test_imgs / 255.0
val_imgs = val_imgs / 255.0

labels = np.load('75/labels.npy')
labels = labels.astype('int32')
labels = labels[indices]
train_labels = labels[:split_1]
val_labels = labels[split_1:split_2]
test_labels = labels[split_2:]

train_imgs = train_imgs.reshape((train_imgs.shape[0], 75, 75, 1))
val_imgs = val_imgs.reshape((val_imgs.shape[0], 75, 75, 1))
test_imgs = test_imgs.reshape((test_imgs.shape[0], 75, 75, 1))

## Classification Model

In [73]:

def conv_time(time):
    ntime = 0
    if time[1] > 30:
        ntime = (time[0] + 0.5)
    else:
        ntime = time[0]
    return ntime

train_labels_converted = np.array([conv_time(time) for time in train_labels])
test_labels_converted = np.array([conv_time(time) for time in test_labels])
val_labels_converted = np.array([conv_time(time) for time in val_labels])

encoder = LabelEncoder()
test_labels_encoded = encoder.fit_transform(test_labels_converted.reshape(-1))
train_labels_encoded = encoder.fit_transform(train_labels_converted.reshape(-1))
val_labels_encoded = encoder.fit_transform(val_labels_converted.reshape(-1))

OHencoder = OneHotEncoder(sparse_output=False)
train_labels_oh = OHencoder.fit_transform(train_labels_encoded.reshape(-1, 1))
val_labels_oh = OHencoder.fit_transform(val_labels_encoded.reshape(-1, 1))

In [74]:
input_shape = (75, 75, 1)
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(kernel_size=(5,5), strides = (2,2), activation="relu", filters=32))
model.add(keras.layers.Conv2D(activation="relu", filters=32, kernel_size=(3,3), input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=32))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))
model.add(keras.layers.Conv2D(kernel_size=(3,3), activation="relu", filters=64))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=512, activation="relu"))
model.add(keras.layers.Dense(units=512, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=256, activation="relu"))
model.add(keras.layers.Dense(units=128, activation="relu"))
model.add(keras.layers.Dense(units=64, activation="relu"))
model.add(keras.layers.Dense(units=32, activation="relu"))
model.add(keras.layers.Dense(units=24, activation="softmax"))

optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

print(train_imgs.shape, train_labels_oh.shape)
val_labels_oh = val_labels_oh.reshape((val_labels_oh.shape[0], 24))

model.fit(train_imgs, train_labels_oh, epochs=10, batch_size=256, validation_data=(val_imgs, val_labels_oh), callbacks=[early_stop])

(14400, 75, 75, 1) (14400, 24)
Epoch 1/10
57/57 [==============================] - 14s 172ms/step - loss: 3.1781 - accuracy: 0.0407 - val_loss: 3.1782 - val_accuracy: 0.0339
Epoch 2/10
57/57 [==============================] - 9s 160ms/step - loss: 3.1780 - accuracy: 0.0417 - val_loss: 3.1782 - val_accuracy: 0.0378
Epoch 3/10
21/57 [==========>...................] - ETA: 5s - loss: 3.1778 - accuracy: 0.0443

KeyboardInterrupt: 

In [76]:
preds = model.predict(test_imgs)
preds = np.argmax(preds, axis=1)

results = encoder.inverse_transform(preds)

accuracy = np.sum(results == test_labels_converted) / len(test_labels_converted)
print(accuracy*100, '%')

169/169 [==============================] - 1s 6ms/step
4.018518518518518 %


## Regression Model

In [96]:
import keras.backend as K

#returning the common sense difference between two times
def difference_func(true, pred):
    diff_1 = K.abs(true - pred)
    diff_2 = K.abs(true - pred + 12*60)

    return K.minimum(diff_1, diff_2)

In [97]:
train_labels_reg = np.array([(time[0]*60+time[1]) for time in train_labels])
test_labels_reg = np.array([(time[0]*60+time[1]) for time in test_labels])
val_labels_reg = np.array([(time[0]*60+time[1]) for time in val_labels])


model = keras.models.Sequential()
model.add(keras.layers.Conv2D(activation='relu', filters=32, kernel_size=(3,3), strides = (2,2),input_shape=(75, 75, 1)))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.Conv2D(filters=32 ,kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))


model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=1, activation="softplus"))
model.compile(loss="mse", optimizer="adam", metrics=[difference_func])

early_stop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)

model.fit(train_imgs, train_labels_reg, epochs=40, batch_size = 512, validation_data = (val_imgs, val_labels_reg), callbacks = [early_stop])

Epoch 1/40
29/29 [==============================] - 9s 177ms/step - loss: 98222.2734 - difference_func: 249.6532 - val_loss: 43600.2344 - val_difference_func: 180.6982
Epoch 2/40
29/29 [==============================] - 5s 161ms/step - loss: 44644.5039 - difference_func: 180.9264 - val_loss: 44855.0195 - val_difference_func: 182.8785
Epoch 3/40
29/29 [==============================] - 5s 169ms/step - loss: 43756.0625 - difference_func: 180.0819 - val_loss: 43436.1953 - val_difference_func: 180.7289
Epoch 4/40
29/29 [==============================] - 5s 169ms/step - loss: 43495.2773 - difference_func: 180.0190 - val_loss: 43397.1562 - val_difference_func: 180.6951
Epoch 5/40
29/29 [==============================] - 5s 170ms/step - loss: 43383.8516 - difference_func: 179.9154 - val_loss: 43382.0938 - val_difference_func: 180.7759
Epoch 6/40
29/29 [==============================] - 5s 163ms/step - loss: 43644.4766 - difference_func: 180.2017 - val_loss: 43478.9023 - val_difference_func: 1

KeyboardInterrupt: 

In [ ]:
reg_preds = model.predict(test_imgs)
reg_preds = [difference_func(pred, test_labels_reg[i]) for i, pred in enumerate(reg_preds)]
reg_preds = np.array(reg_preds)


169/169 [==============================] - 2s 9ms/step
[[350.99246]
 [352.9995 ]
 [354.80753]
 ...
 [354.764  ]
 [351.82877]
 [353.31216]]
[713 479 155 ... 267 311 169]


2.9653396429368004